In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA silver;

## Modelagem

A partir da tabela voos iremos montar nosso esquema estrela. O esquema será composto de 5 dimensões e um fato:

#### Dimensão aeronave:
- **id_equipamento (chave primária)**
- sg_equipamento_icao
- ds_modelo
- ds_matricula

#### Dimensão aeroporto:
- **id_aerodromo (chave primária)**
- id_aerodromo
- sg_icao
- sg_iata
- nm_aerodromo
- nm_municipio
- sg_uf
- nm_regiao
- nm_pais
- nm_continente

#### Dimensão empresa:
- **id_empresa (chave primária)**
- sg_empresa_icao
- sg_empresa_iata
- nm_empresa
- nm_pais
- ds_tipo_empresa

#### Dimensão linha:
- **id_tipo_linha (chave primária)**
- cd_tipo_linha
- ds_tipo_linha
- ds_natureza_tipo_linha
- ds_servico_tipo_linha

#### Dimensão tempo:
- **dt_voo (chave primária)**
- nr_dia
- nr_mes
- nr_ano
- nr_semestre
- nr_trimestre intege

## Aplicando filtros e critérios de qualidade

Para entender melhor o motivo dessas operações, consulte esta planilha [aqui](https://docs.google.com/spreadsheets/d/1_4_-XutVG2gyGrHMkfTlin1-UbQRA27tdRVC9ZI0fmk/edit?usp=sharing). 
1. Selecionar colunas necessárias (consultar motivo para colunas serem descartadas na planilha a cima);
2. Aplicar filtros e correção de valores inválidos;
3. Aplicar tipagem correta nas colunas;
4. Montar entidades dimensões;
5. Montar entidade fato;


In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.table("mvp.bronze.voos")

### Seleção das colunas necessárias:

In [0]:
df_colunas = df.select([
  "id_equipamento",
  "sg_equipamento_icao",
  "ds_modelo",
  "ds_matricula",
  "id_aerodromo_origem",
  "sg_icao_origem",
  "sg_iata_origem",
  "nm_aerodromo_origem",
  "nm_municipio_origem",
  "sg_uf_origem",
  "nm_regiao_origem",
  "nm_pais_origem",
  "nm_continente_origem",
  "id_aerodromo_destino",
  "sg_icao_destino",
  "sg_iata_destino",
  "nm_aerodromo_destino",
  "nm_municipio_destino",
  "sg_uf_destino",
  "nm_regiao_destino",
  "nm_pais_destino",
  "nm_continente_destino",
  "dt_partida_real",
  "id_empresa",
  "sg_empresa_icao",
  "sg_empresa_iata",
  "nm_empresa",
  "nm_pais",
  "id_tipo_linha",
  "cd_tipo_linha",
  "ds_tipo_linha",
  "ds_natureza_tipo_linha",
  "ds_servico_tipo_linha",
  "id_basica",
  "nr_voo",
  "nr_singular",
  "ds_natureza_etapa",
  "dt_chegada_real",
  "lt_combustivel",
  "nr_assentos_ofertados",
  "kg_payload",
  "km_distancia",
  "nr_passag_pagos",
  "nr_passag_gratis",
  "kg_bagagem_livre",
  "kg_bagagem_excesso",
  "kg_carga_paga",
  "kg_carga_gratis",
  "kg_correio",
  "nr_horas_voadas",
  "kg_peso",
  "nr_velocidade_media",
  "nr_pax_gratis_km",
  "nr_carga_paga_km",
  "nr_carga_gratis_km",
  "nr_correio_km",
  "nr_bagagem_paga_km",
  "nr_bagagem_gratis_km",
  "nr_ask",
  "nr_rpk",
  "nr_atk"
])

### Aplicação de filtros e correção de valores inválidos:

In [0]:
print("Excluindo linhas nulas:")
print("="*20)
colunas = [
  "id_equipamento",
  "id_aerodromo_origem", 
  "id_aerodromo_destino",
  "id_empresa",
  "id_tipo_linha", 
  "id_basica", 
]
df_nao_nulos = df_colunas.selectExpr("*")
for coluna in colunas:
  print("Total de linhas antes da exclusão: ", df_nao_nulos.count())
  
  total_registros_nulos = df_nao_nulos.where(F.col(coluna).isNull()).count()
  print("Total de linhas nulas na coluna ", coluna, ": ", total_registros_nulos)
  
  df_nao_nulos = df_nao_nulos.filter(F.col(coluna).isNotNull())

  print("Total de linhas depois da exclusão: ", df_nao_nulos.count())
  print("-"*20)

In [0]:
print("Convertendo strings vazias em null:")
print("="*20)
colunas = [
  "ds_modelo",
  "ds_matricula",
  "nm_aerodromo_origem",
  "nm_municipio_origem",
  "nm_regiao_origem",
  "nm_pais_origem",
  "nm_continente_origem",
  "nm_aerodromo_destino",
  "nm_municipio_destino",
  "nm_regiao_destino",
  "nm_pais_destino",
  "nm_continente_destino",
  "nm_empresa",
  "nm_pais",
  "ds_tipo_linha",
  "ds_natureza_tipo_linha",
  "nr_singular"
]
df_sem_vazios = df_nao_nulos.selectExpr("*")
for coluna in colunas:
  print(f"Total de linhas da coluna {coluna} com strings vazias ANTES do processamento: ", df_sem_vazios.where(F.trim(F.col(coluna)) == "").count())
  df_sem_vazios = df_sem_vazios.withColumn(coluna, F.when((F.trim(F.col(coluna)) == ""), F.lit(None)).otherwise(F.trim(F.col(coluna))))
  print(f"Total de linhas da coluna {coluna} com strings vazias DEPOIS do processamento: ", df_sem_vazios.where(F.trim(F.col(coluna)) == "").count())
  print("-"*20)


In [0]:
print("Removendo valores inteiros menores que 0:")
print("="*20)
from pyspark.sql.types import IntegerType
colunas = [
    "nr_voo",
    "lt_combustivel",
    "nr_assentos_ofertados",
    "kg_payload",
    "km_distancia",
    "nr_passag_pagos",
    "nr_passag_gratis",
    "kg_bagagem_livre",
    "kg_bagagem_excesso",
    "kg_carga_paga",
    "kg_carga_gratis",
    "kg_correio",
    "kg_peso",
    "nr_pax_gratis_km",
    "nr_carga_paga_km",
    "nr_carga_gratis_km",
    "nr_correio_km",
    "nr_bagagem_paga_km",
    "nr_bagagem_gratis_km",
    "nr_ask",
    "nr_rpk",
    "nr_atk"
]
df_sem_inteiros_menores_zero = df_sem_vazios.selectExpr("*")
for coluna in colunas:
    df_sem_inteiros_menores_zero = df_sem_inteiros_menores_zero.withColumn(coluna, F.col(coluna).try_cast(IntegerType()))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_inteiros_menores_zero.where(F.col(coluna) < 0 ).count())
    df_sem_inteiros_menores_zero = df_sem_inteiros_menores_zero.withColumn(coluna, F.when((F.col(coluna) < 0), F.lit(None)).otherwise(F.col(coluna)))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_inteiros_menores_zero.where(F.col(coluna) < 0 ).count())
    print("-"*20)

In [0]:
print("Removendo valores float menores que 0:")
print("="*20)
from pyspark.sql.types import FloatType
colunas = [
    "nr_horas_voadas",
    "nr_velocidade_media"
]
df_sem_float_menores_zero = df_sem_vazios.selectExpr("*")
for coluna in colunas:
    df_sem_float_menores_zero = df_sem_float_menores_zero.withColumn(coluna, F.col(coluna).try_cast(FloatType()))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_float_menores_zero.where(F.col(coluna) < 0 ).count())
    df_sem_float_menores_zero = df_sem_float_menores_zero.withColumn(coluna, F.when((F.col(coluna) < 0), F.lit(None)).otherwise(F.col(coluna)))
    print(f"Total de valores menores que zero na coluna {coluna} ANTES do processamento: ", df_sem_float_menores_zero.where(F.col(coluna) < 0 ).count())
    print("-"*20)

In [0]:
df_valores_validos = df_sem_float_menores_zero.selectExpr("*")
#Valores válidos:
coluna = "cd_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = ["N", "C", "I", "G", "X"]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("X")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


In [0]:
coluna = "ds_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = [
    "DOMÉSTICA MISTA",
    "DOMÉSTICA CARGUEIRA",
    "INTERNACIONAL CARGUEIRA",
    "INTERNACIONAL MISTA",
    "NÃO IDENTIFICADA"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADA")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


In [0]:
coluna = "ds_natureza_etapa"
print("Processamento da coluna:", coluna)
valores_validos = [
    "DOMÉSTICA",
    "INTERNACIONAL",
    "NÃO IDENTIFICADA"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADA")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


In [0]:
coluna = "ds_servico_tipo_linha"
print("Processamento da coluna:", coluna)
valores_validos = [
    "NÃO IDENTIFICADO",
    "PASSAGEIRO",
    "CARGUEIRO"
]
print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())

df_valores_validos = df_valores_validos.withColumn(coluna, F.when(~(F.trim(F.col(coluna)).isin(valores_validos)), F.lit("NÃO IDENTIFICADO")).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos.where(~(F.trim(F.col(coluna)).isin(valores_validos))).count())


In [0]:
coluna = "dt_partida_real"
print("Processamento da coluna:", coluna)

df_valores_validos = df_valores_validos.withColumn(coluna, F.try_to_date(F.col(coluna), "yyyy-MM-dd"))

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())

df_valores_validos = df_valores_validos.filter(F.col(coluna).isNotNull())

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())



In [0]:
coluna = "dt_chegada_real"
print("Processamento da coluna:", coluna)

df_valores_validos = df_valores_validos.withColumn(coluna, F.try_to_date(F.col(coluna), "yyyy-MM-dd"))

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())

df_valores_validos = df_valores_validos.filter(F.col(coluna).isNotNull())

print("Total de valores inválidos ANTES do processamento:", df_valores_validos.where(F.col(coluna).isNull()).count())



In [0]:
from pyspark.sql.types import StringType
df_valores_validos_len = df_valores_validos.selectExpr("*")
colunas_len = {
    "sg_uf_origem": 2,
    "sg_uf_destino": 2,
    "sg_empresa_iata": 2,
    "sg_iata_origem": 3,
    "sg_iata_destino": 3,
    "sg_empresa_icao": 3,
    "sg_icao_origem": 4,
    "sg_icao_destino": 4
}
for coluna, tamanho in colunas_len.items():
    print("Processamento da coluna:", coluna)
    print("Tamanho válido:", tamanho)

    print("Total de valores inválidos ANTES do processamento:", df_valores_validos_len.where(~(F.length(F.trim(F.col(coluna))) == tamanho)).count())

    df_valores_validos_len = df_valores_validos_len.withColumn(coluna, F.when(~(F.length(F.trim(F.col(coluna))) == tamanho), F.lit(None)).otherwise(F.trim(F.col(coluna))))

    print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos_len.where(~(F.length(F.trim(F.col(coluna))) == tamanho)).count())

In [0]:
coluna = "sg_equipamento_icao"
print("Processamento da coluna:", coluna)
min = 2
max = 4


print("Total de valores inválidos ANTES do processamento:", df_valores_validos_len.where(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max))).count())

df_valores_validos_len = df_valores_validos_len.withColumn(coluna, F.when(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max)), F.lit(None)).otherwise(F.trim(F.col(coluna))))

print("Total de valores inválidos DEPOIS do processamento:", df_valores_validos_len.where(~((F.length(F.trim(F.col(coluna))) >= min) & (F.length(F.trim(F.col(coluna))) <= max))).count())

### Aplicação da tipagem nas colunas:

In [0]:
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType 
df = df_valores_validos_len.selectExpr("*")
df_tipado = df.select(
    F.col("cd_tipo_linha").cast(StringType()),
    F.col("ds_matricula").cast(StringType()),
    F.col("ds_modelo").cast(StringType()),
    F.col("ds_natureza_etapa").cast(StringType()),
    F.col("ds_natureza_tipo_linha").cast(StringType()),
    F.col("ds_servico_tipo_linha").cast(StringType()),
    F.col("ds_tipo_linha").cast(StringType()),
    F.col("nm_aerodromo_destino").cast(StringType()),
    F.col("nm_aerodromo_origem").cast(StringType()),
    F.col("nm_continente_destino").cast(StringType()),
    F.col("nm_continente_origem").cast(StringType()),
    F.col("nm_empresa").cast(StringType()),
    F.col("nm_municipio_destino").cast(StringType()),
    F.col("nm_municipio_origem").cast(StringType()),
    F.col("nm_pais").cast(StringType()),
    F.col("nm_pais_destino").cast(StringType()),
    F.col("nm_pais_origem").cast(StringType()),
    F.col("nm_regiao_destino").cast(StringType()),
    F.col("nm_regiao_origem").cast(StringType()),
    F.col("nr_singular").cast(StringType()),
    F.col("sg_empresa_iata").cast(StringType()),
    F.col("sg_empresa_icao").cast(StringType()),
    F.col("sg_equipamento_icao").cast(StringType()),
    F.col("sg_iata_destino").cast(StringType()),
    F.col("sg_iata_origem").cast(StringType()),
    F.col("sg_icao_destino").cast(StringType()),
    F.col("sg_icao_origem").cast(StringType()),
    F.col("sg_uf_destino").cast(StringType()),
    F.col("sg_uf_origem").cast(StringType()),

    F.col("id_aerodromo_destino").cast(IntegerType()),
    F.col("id_aerodromo_origem").cast(IntegerType()),
    F.col("id_basica").cast(IntegerType()),
    F.col("id_empresa").cast(IntegerType()),
    F.col("id_equipamento").cast(IntegerType()),
    F.col("id_tipo_linha").cast(IntegerType()),
    F.col("kg_bagagem_excesso").cast(IntegerType()),
    F.col("kg_bagagem_livre").cast(IntegerType()),
    F.col("kg_carga_gratis").cast(IntegerType()),
    F.col("kg_carga_paga").cast(IntegerType()),
    F.col("kg_correio").cast(IntegerType()),
    F.col("kg_payload").cast(IntegerType()),
    F.col("kg_peso").cast(IntegerType()),
    F.col("km_distancia").cast(IntegerType()),
    F.col("lt_combustivel").cast(IntegerType()),
    F.col("nr_ask").cast(IntegerType()),
    F.col("nr_assentos_ofertados").cast(IntegerType()),
    F.col("nr_atk").cast(IntegerType()),
    F.col("nr_bagagem_gratis_km").cast(IntegerType()),
    F.col("nr_bagagem_paga_km").cast(IntegerType()),
    F.col("nr_carga_gratis_km").cast(IntegerType()),
    F.col("nr_carga_paga_km").cast(IntegerType()),
    F.col("nr_correio_km").cast(IntegerType()),
    F.col("nr_passag_gratis").cast(IntegerType()),
    F.col("nr_passag_pagos").cast(IntegerType()),
    F.col("nr_pax_gratis_km").cast(IntegerType()),
    F.col("nr_rpk").cast(IntegerType()),
    F.col("nr_voo").cast(IntegerType()),

    F.col("nr_horas_voadas").cast(FloatType()),
    F.col("nr_velocidade_media").cast(FloatType()),

    F.col("dt_chegada_real").cast(DateType()),
    F.col("dt_partida_real").cast(DateType())
)

In [0]:
df_tipado.printSchema()

In [0]:
df_tipado.write.format("delta").mode("overwrite").saveAsTable("voos_tipado")

### Construção das entidades dimensões:

In [0]:
df = spark.sql("select * from voos_tipado")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

def moda_por_coluna(df, chave, coluna):
    w = Window.partitionBy(chave).orderBy(F.desc("count"))
    
    return (
        df.groupBy(chave, coluna)
          .count()
          .withColumn("rn", F.row_number().over(w))
          .filter("rn = 1")
          .select(chave, coluna)
    )

In [0]:
# dim_aeronave
print("dim_aeronave:")
column_id = "id_equipamento"
colunas = ["ds_modelo", "ds_matricula", "sg_equipamento_icao"]
dim_aeronave = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_aeronave = moda_df if dim_aeronave is None else dim_aeronave.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeronave.groupBy(column_id).count().where("count > 1").count())
dim_aeronave.limit(5).display()


In [0]:

# dim_aeroporto
print("dim_aeroporto:")
aeroportos = df.select([
    F.col("id_aerodromo_destino").alias("id_aerodromo"),
    F.col("nm_aerodromo_destino").alias("nm_aerodromo"),
    F.col("nm_continente_destino").alias("nm_continente"),
    F.col("nm_municipio_destino").alias("nm_municipio"),
    F.col("nm_pais_destino").alias("nm_pais"),
    F.col("nm_regiao_destino").alias("nm_regiao"),
    F.col("sg_iata_destino").alias("sg_iata"),
    F.col("sg_icao_destino").alias("sg_icao"),
    F.col("sg_uf_destino").alias("sg_uf")
]).union(df.select([
    F.col("id_aerodromo_origem").alias("id_aerodromo"),
    F.col("nm_aerodromo_origem").alias("nm_aerodromo"),
    F.col("nm_continente_origem").alias("nm_continente"),
    F.col("nm_municipio_origem").alias("nm_municipio"),
    F.col("nm_pais_origem").alias("nm_pais"),
    F.col("nm_regiao_origem").alias("nm_regiao"),
    F.col("sg_iata_origem").alias("sg_iata"),
    F.col("sg_icao_origem").alias("sg_icao"),
    F.col("sg_uf_origem").alias("sg_uf")
]))
column_id = "id_aerodromo"
colunas = [
    "nm_aerodromo",
    "nm_continente",
    "nm_municipio",
    "nm_pais",
    "nm_regiao",
    "sg_iata",
    "sg_icao",
    "sg_uf"
]
dim_aeroporto = None

for col in colunas:
    moda_df = moda_por_coluna(aeroportos, column_id, col)
    dim_aeroporto = moda_df if dim_aeroporto is None else dim_aeroporto.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeroporto.groupBy(column_id).count().where("count > 1").count())
dim_aeroporto.limit(5).display()


In [0]:
# dim_empresa
print("dim_empresa:")
column_id = "id_empresa"
colunas = [
    "sg_empresa_icao",
    "sg_empresa_iata",
    "nm_empresa",
    "nm_pais"
] 
  
dim_empresa = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_empresa = moda_df if dim_empresa is None else dim_empresa.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_empresa.groupBy(column_id).count().where("count > 1").count())
dim_empresa.limit(5).display()


In [0]:
# dim_linha
print("dim_linha:")
column_id = "id_tipo_linha"
colunas = [
    "cd_tipo_linha", 
    "ds_tipo_linha", 
    "ds_natureza_tipo_linha", 
    "ds_servico_tipo_linha"
] 
  
dim_linha = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_linha = moda_df if dim_linha is None else dim_linha.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_linha.groupBy(column_id).count().where("count > 1").count())
dim_linha.limit(5).display()


In [0]:
# dim_data
datas = df.select([
    F.col("dt_partida_real").alias("dt_voo")
]).union(df.select([
    F.col("dt_chegada_real").alias("dt_voo")
]))

column_id = "dt_voo"
datas = datas.distinct()
dim_data = datas\
    .withColumn("dt_voo", F.to_date(F.col("dt_voo")))\
    .withColumn("nr_dia", F.day("dt_voo"))\
    .withColumn("nr_mes", F.month("dt_voo"))\
    .withColumn("nr_ano", F.year("dt_voo"))\
    .withColumn("nr_semestre", F.when(F.col("nr_mes") <= 6, 1).otherwise(2))\
    .withColumn("nr_trimestre", F.ceil(F.col("nr_mes") / 3))

print("IDs repetidos:", dim_data.groupBy(column_id).count().where("count > 1").count())
dim_data.limit(5).display()
    

### Construção da tabela fato:

In [0]:
column_id = "id_basica"
fact_voo = df.select([
  "id_basica",
  "nr_voo",
  "nr_singular",
  "ds_natureza_etapa",
  "lt_combustivel", 
  "nr_assentos_ofertados",
  "kg_payload",
  "km_distancia",
  "nr_passag_pagos",
  "nr_passag_gratis", 
  "kg_bagagem_livre",
  "kg_bagagem_excesso",
  "kg_carga_paga",
  "kg_carga_gratis",
  "kg_correio",
  "nr_horas_voadas",
  "kg_peso",
  "nr_velocidade_media",
  "nr_pax_gratis_km",
  "nr_carga_paga_km",
  "nr_carga_gratis_km",
  "nr_correio_km",
  "nr_bagagem_paga_km",
  "nr_bagagem_gratis_km",
  "nr_ask",
  "nr_rpk",
  "nr_atk",
  "dt_partida_real",
  "dt_chegada_real",
  "id_tipo_linha",
  "id_empresa",
  "id_aerodromo_origem",
  "id_aerodromo_destino",
  "id_equipamento",
])


fact_voo = fact_voo.withColumnRenamed("id_aerodromo_origem", "pk_id_aerodromo_partida")
fact_voo = fact_voo.withColumnRenamed("id_aerodromo_destino", "pk_id_aerodromo_chegada")
fact_voo = fact_voo.withColumnRenamed("id_equipamento", "pk_id_equipamento")
fact_voo = fact_voo.withColumnRenamed("id_empresa", "pk_id_empresa")
fact_voo = fact_voo.withColumnRenamed("id_tipo_linha", "pk_id_tipo_linha")
fact_voo = fact_voo.withColumnRenamed("dt_partida_real", "pk_id_dt_partida")
fact_voo = fact_voo.withColumnRenamed("dt_chegada_real", "pk_id_dt_chegada")

print("IDs repetidos:", fact_voo.groupBy(column_id).count().where("count > 1").count())
fact_voo.limit(5).display()


In [0]:
print("Total de registros na tabela dim_aeronave", dim_aeronave.count())
print("Total de registros na tabela dim_aeroporto", dim_aeroporto.count())
print("Total de registros na tabela dim_empresa", dim_empresa.count())
print("Total de registros na tabela dim_linha", dim_linha.count())
print("Total de registros na tabela dim_data", dim_data.count())
print("Total de registros na tabela fact_voo", fact_voo.count())

### Persistência das tabelas:

In [0]:
dim_aeronave.write.format("delta").mode("overwrite").saveAsTable("dim_aeronave")
dim_aeroporto.write.format("delta").mode("overwrite").saveAsTable("dim_aeroporto")
dim_empresa.write.format("delta").mode("overwrite").saveAsTable("dim_empresa")
dim_linha.write.format("delta").mode("overwrite").saveAsTable("dim_linha")
dim_data.write.format("delta").mode("overwrite").saveAsTable("dim_data")
fact_voo.write.format("delta").mode("overwrite").saveAsTable("fact_voo")